In [1]:
import cv2 #openCV
import time

In [2]:
# pip install urllib
# pip install m3u8
# pip install streamlink
import urllib
import m3u8
import streamlink
import cv2 #openCV


In [5]:
#Ref: https://stackoverflow.com/questions/55631634/recording-youtube-live-stream-to-file-in-python
def get_stream(url):
    """
    Get upload chunk url
    input: youtube URL
    output: m3u8 object segment
    """
    #Get stream
    streams = streamlink.streams(url)
    stream_url = streams["360p"] #Alternate, use "best"

    m3u8_obj = m3u8.load(stream_url.args['url'])
    return m3u8_obj.segments[0] #Parsed stream


def dl_stream(url, filename, chunks):
    """
    Download each chunk to file
    input: url, filename, and number of chunks (int)
    output: saves file at filename location
    returns none.
    """
    pre_time_stamp = 0
    
    #Repeat for each chunk
    #Needs to be in chunks beceause 
    #  1) it's live and 
    #  2) it won't let you leave the stream open forever
    i=1
    while i <= chunks:
       
        #Open stream
        stream_segment = get_stream(url)
        #Get current time
        cur_time_stamp = \
            stream_segment.program_date_time.strftime("%Y%m%d-%H%M%S")
        #Only get next time step, wait if it's not new yet
        if pre_time_stamp == cur_time_stamp:
            #Don't increment counter if we didn't get a chunk
            pass
        else:
            print(f'#{i} at time {cur_time_stamp}')
            #Open file for writing stream
            file = open(filename, 'ab+') #ab+ means keep adding to file
            #Write stream to file
            with urllib.request.urlopen(stream_segment.uri) as response:
                html = response.read()
                file.write(html)
            #Update time stamp
            pre_time_stamp = cur_time_stamp
            i += 1 #only increment if we got a new chunk

    return None

def openCVProcessing(saved_video_file):
    '''View saved video with openCV
    Add your other steps here'''
    capture = cv2.VideoCapture(saved_video_file)

    while capture.isOpened():
        grabbed, frame = capture.read()  #read in single frame
        if grabbed == False:
            break

        #openCV processing goes here
        #
        
        cv2.imshow('frame',frame)  #Show the frame
        
        #Shown in a new window, To exit, push q on the keyboard
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break

    capture.release()
    cv2.destroyAllWindows()  #close the windows automatically
                  
tempFile = "temp.ts"  #files are format ts, open cv can view them
videoURL = "https://www.youtube.com/watch?v=DoUOrTJbIu4" #Jackson hole town square, live stream

dl_stream(videoURL, tempFile, 3)
openCVProcessing(tempFile)

#1 at time 20210512-204009
#2 at time 20210512-204014
#3 at time 20210512-204019


In [52]:
def openCVProcessing(saved_video_file):
    #View saved video with openCV
    capture = cv2.VideoCapture(saved_video_file)

    while capture.isOpened():
        grabbed, frame = capture.read()  #read in single frame
        if grabbed == False:
            break

        #openCV processing goes here
        
        
        cv2.imshow('frame',frame)  #Show the frame
        
        #Shown in a new window, To exit, push q on the keyboard
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break

    capture.release()
    cv2.destroyAllWindows()  #close the windows automatically


In [46]:
videoURL = "https://www.youtube.com/watch?v=DoUOrTJbIu4"
stream_segment = get_stream(videoURL)

# Attempt to combine streaming and working into one!

In [3]:
#Ref: https://stackoverflow.com/questions/55631634/recording-youtube-live-stream-to-file-in-python
def get_stream(url):
    """
    Get upload chunk url
    input: youtube URL
    output: m3u8 object segment
    """
    #Get stream
    streams = streamlink.streams(url)
    stream_url = streams["360p"] #Alternate, use "best"

    m3u8_obj = m3u8.load(stream_url.args['url'])
    return m3u8_obj.segments[0] #Parsed stream

def openCVProcessing(saved_video_file):
    '''View saved video with openCV
    Add your other steps here'''
    capture = cv2.VideoCapture(saved_video_file)

    while capture.isOpened():
        grabbed, frame = capture.read()  #read in single frame
        if grabbed == False:
            break

        #openCV processing goes here
        #
        
        cv2.imshow('frame',frame)  #Show the frame
        
        #Shown in a new window, To exit, push q on the keyboard
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break

    capture.release()
    cv2.destroyAllWindows()  #close the windows automatically

def dl_stream(url, filename, chunks):
    """
    Download each chunk to file
    input: url, filename, and number of chunks (int)
    output: saves file at filename location
    returns none.
    """
    pre_time_stamp = 0
    
    #Repeat for each chunk
    #Needs to be in chunks beceause 
    #  1) it's live and 
    #  2) it won't let you leave the stream open forever
    i=1
    while i <= chunks:
       
        #Open stream
        stream_segment = get_stream(url)
        #Get current time
        cur_time_stamp = \
            stream_segment.program_date_time.strftime("%Y%m%d-%H%M%S")
        #Only get next time step, wait if it's not new yet
        if pre_time_stamp == cur_time_stamp:
            #Don't increment counter if we didn't get a chunk
            pass
        else:
            print(f'#{i} at time {cur_time_stamp}')
            #Open file for writing stream
            file = open(filename, 'ab+') #ab+ means keep adding to file
            #Write stream to file
            with urllib.request.urlopen(stream_segment.uri) as response:
                html = response.read()
                openCVProcessing(html)
                #file.write(html)
            #Update time stamp
            pre_time_stamp = cur_time_stamp
            i += 1 #only increment if we got a new chunk

    return None


tempFile = "temp.ts"  #files are format ts, open cv can view them
videoURL = "https://www.youtube.com/watch?v=DoUOrTJbIu4" #Jackson hole town square, live stream

dl_stream(videoURL, tempFile, 1)
#openCVProcessing(tempFile)

#1 at time 20210512-203714


TypeError: an integer is required (got type bytes)

In [10]:
url = "https://www.youtube.com/watch?v=DoUOrTJbIu4" #Jackson hole town square, live stream

stream_segment = get_stream(url)
#Open file for writing stream
file = open("test.ts", 'ab+') #ab+ means keep adding to file
#Write stream to file
with urllib.request.urlopen(stream_segment.uri) as response:
    html = response.read()
    #openCVProcessing(html)
    file.write(html)
    openCVProcessing("test.ts")